<a href="https://colab.research.google.com/github/siddhesh-k/Free-QR-Code-Generator-App/blob/main/Deepfake_end_to_end_tortoise_wav2lip_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Automatic DeepFake Creation (Tortoise voice cloning + wav2lip)

Note: A video is required. If an audio is also provided, the voice will be cloned from the audio.

# **The video should have a face looking at all times to the camera.**

## **Video (and audio) file should be in Google drive in a folder named 'deepfake'. No other files should exist there**

wav2lip code taken from https://github.com/snehitvaddi/Deepfake-using-Wave2Lip

In [1]:
#@title Upload video.mp4 (video to overlay voice) & voice.mp3 (voice to clone) files - Should be mp3 and mp4, having any name
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
cd /content/gdrive/MyDrive/deepfake

/content/gdrive/MyDrive/deepfake


In [3]:
base_path='/content/gdrive/MyDrive/deepfake' #Specify path of video/audio

In [4]:
#@title Install TTS, pydub to create folders with audio chunks, and moviepy to modify duration of audio/video
!pip uninstall -y numpy scipy librosa TTS
!pip install scipy librosa TTS soundfile
!pip install -q pydub moviepy==1.0.3

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scipy 1.14.1
Uninstalling scipy-1.14.1:
  Successfully uninstalled scipy-1.14.1
Found existing installation: librosa 0.11.0
Uninstalling librosa-0.11.0:
  Successfully uninstalled librosa-0.11.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 78.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 129.3 MB/s eta 0:00:00
  Preparing metadata (set

In [2]:
#@title English text that we want to read with the cloned voice - This will be inserted in the video too
#Text prompt should be separated with '|' every one to two sentences (every ~20secs it takes to read it).
text_to_read="""Joining two modalities results in a surprising increase in generalization! |
                What would happen if we combined them all? """

In [16]:
#@title Rename audio and video files to be used below
#Prompt: the user selects and uploads to google colab one audio and one video file. Rename the audio file to 'input_voice.mp3' and the video to 'input_video.mp4'
import os

# Loop over files in the directory
for file in os.listdir(os.getcwd()):

      filename = os.path.join(base_path, file)

      if filename.endswith('.mp3'):
          new_filename = 'input_voice.mp3'
          os.rename(filename, new_filename)
      if filename.endswith('.mp4'):
          new_filename = 'video_full.mp4'
          os.rename(filename, new_filename)

#If only video is provided:
from moviepy.editor import VideoFileClip

def extract_audio(input_video, output_audio):
    video = VideoFileClip(input_video)
    audio = video.audio
    audio.write_audiofile(output_audio)

# Provide the input video file path and desired output audio file path
input_video = 'video_full.mp4'
output_audio = 'input_voice.mp3'

#Decide if voice will be cloned from video or audio
mp3_check=0
for file in os.listdir(os.getcwd()):
      file_path =  os.path.join(base_path, file)
      if '.mp3' in file_path:
        mp3_check=1

if mp3_check==0:
  print("Voice will be cloned from video")
  extract_audio(input_video, output_audio)

Voice will be cloned from video


OSError: MoviePy error: the file video_full.mp4 could not be found!
Please check that you entered the correct path.

In [17]:
#@title Create folder with 10 secs chunks of audio to be used as input in Tortoise
from pydub import AudioSegment

def split_audio_to_clips(audio_file, output_dir, clip_length=10000, sample_rate=22050):
    # Load the audio file
    audio = AudioSegment.from_mp3(audio_file)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Calculate the total number of clips
    num_clips = len(audio) // clip_length

    # Split the audio into clips and save them as WAV files
    for i in range(num_clips):
        start_time = i * clip_length
        end_time = start_time + clip_length
        clip = audio[start_time:end_time]

        # Set the sample width to 2 bytes for floating-point format
        clip = clip.set_sample_width(2)

        # Set the sample rate to 22050 Hz
        clip = clip.set_frame_rate(sample_rate)

        # Save the clip as a WAV file
        clip.export(os.path.join(output_dir, f"{i+1}.wav"), format="wav")

    print(f"{num_clips} clips saved in '{output_dir}'.")

if __name__ == "__main__":

    # Replace 'input_audio.mp3' with the name of your MP3 file
    input_audio_file = base_path+'/input_voice.mp3'

    # Replace 'voices' with the desired subdirectory name
    subdirectory_name = base_path+'/voice'

    split_audio_to_clips(input_audio_file, subdirectory_name)

0 clips saved in '/content/gdrive/MyDrive/deepfake/voice'.


In [8]:


!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.31.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

fatal: destination path 'tortoise-tts' already exists and is not an empty directory.
/content/gdrive/MyDrive/deepfake/tortoise-tts
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of resampy to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.


In [15]:
#@title Download and run TTS tortoise model
# import librosa
# from TTS.api import TTS
# tts = TTS("tts_models/en/multi-dataset/tortoise-v2")

ModuleNotFoundError: No module named 'numpy.strings'

In [16]:
!pip install numpy==2.0.0
!pip install scipy==1.11.2 librosa==0.9.2 TTS soundfile
!pip install -q pydub moviepy==1.0.3
!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install -U numpy==2.0.0 pandas scikit-learn

import librosa
from TTS.api import TTS
tts = TTS("tts_models/en/multi-dataset/tortoise-v2")

# If the above runs successfully, then proceed with the rest of your code.

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/TorToiSe-2.3.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 74.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

100%|█████████▉| 1.72G/1.72G [01:12<00:00, 46.4MiB/s]
100%|██████████| 1.72G/1.72G [01:12<00:00, 23.6MiB/s]

  1%|          | 8.54M/976M [00:00<00:12, 76.5MiB/s]
  2%|▏         | 16.2M/976M [00:00<00:15, 60.5MiB/s]
  2%|▏         | 22.4M/976M [00:00<00:17, 54.3MiB/s]
  3%|▎         | 29.3M/976M [00:00<00:15, 59.2MiB/s]
  4%|▎         | 35.4M/976M [00:00<00:18, 50.5MiB/s]
  4%|▍         | 41.9M/976M [00:00<00:19, 48.8MiB/s]
  5%|▌         | 52.4M/976M [00:00<00:14, 63.3MiB/s]
  6%|▌         | 60.8M/976M [00:01<00:13, 66.4MiB/s]
  7%|▋         | 67.8M/976M [00:01<00:24, 37.6MiB/s]
  8%|▊         | 73.4M/976M [00:01<00:24, 37.5MiB/s]
  8%|▊         | 81.4M/976M [00:01<00:19, 45.5MiB/s]
  9%|▉         | 87.2M/976M [00:01<00:18, 47.1MiB/s]
 10%|▉         | 93.9M/976M [00:01<00:17, 51.8MiB/s]
 10%|█         | 99.9M/976M [00:02<00:25, 34.5MiB/s]
 11%|█         | 105M/976M [00:02<00:24, 35.4MiB/s] 
 11%|█▏        | 111M/976M [00:02<00:20, 41.6MiB/s]
 12%|█▏        | 117M/976M [00:03<01:03, 13.

 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: tortoise


In [4]:
from tortoise.utils.text import split_and_recombine_text
from time import time

# Process text
text = text_to_read
if '|' in text:
    print("Found the '|' character in your text, which I will use as a cue for where to split it up. If this was not"
          "your intent, please remove all '|' characters from the input.")
    texts = text.split('|')
else:
    texts = split_and_recombine_text(text) #If only one piece of text (<20secs), convert it to list

Found the '|' character in your text, which I will use as a cue for where to split it up. If this was notyour intent, please remove all '|' characters from the input.


In [13]:
cd ..

/content/gdrive/MyDrive


In [6]:
#Tortoise Fastest Inference from script (~2min per 20secs of audio in Colab)
#Can also be downloaded from https://huggingface.co/jbetker/tortoise-tts-v2/tree/main
import torch
import torchaudio

from TTS.tts.configs.tortoise_config import TortoiseConfig
from TTS.tts.models.tortoise import Tortoise
import IPython
import gc

config = TortoiseConfig()
model = Tortoise.init_from_config(config)
model.load_checkpoint(config, checkpoint_dir="/root/.local/share/tts/tts_models--en--multi-dataset--tortoise-v2", eval=True) #Deepspeed doesn't work
model.cuda()

all_parts = []
for j, text in enumerate(texts): #Combine individuals pieces of text by processing sound one piece at a time (around 20secs each)
  # cloning a speaker
  output_dict = model.synthesize(text, config, speaker_id="voice", voice_dirs=base_path)

  #Save result
  torchaudio.save("tortoise_v2_script_"+str(j)+".wav", torch.tensor(output_dict["wav"]).squeeze(0), 24000)
  all_parts.append(torch.tensor(output_dict["wav"]).squeeze(0))
  del output_dict

  # Perform garbage collection
  gc.collect()

torchaudio.save('combined.wav', torch.cat(all_parts, dim=-1), 24000)
IPython.display.Audio('combined.wav')

/usr/local/lib/python3.11/dist-packages/torch/functional.py:709: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /pytorch/aten/src/ATen/native/SpectralOps.cpp:873.)
  return _VF.stft(  # type: ignore[attr-defined]


Generating autoregressive samples..


100%|██████████| 16/16 [00:01<00:00, 11.00it/s]


Transforming autoregressive outputs into audio..


  0%|          | 0/100 [00:00<?, ?it/s]

<ipython-input-6-48ea5f8a04b8>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torchaudio.save("tortoise_v2_script_"+str(j)+".wav", torch.tensor(output_dict["wav"]).squeeze(0), 24000)
<ipython-input-6-48ea5f8a04b8>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_parts.append(torch.tensor(output_dict["wav"]).squeeze(0))


Generating autoregressive samples..


100%|██████████| 16/16 [00:01<00:00,  9.23it/s]


Transforming autoregressive outputs into audio..


  0%|          | 0/100 [00:00<?, ?it/s]

Clean memory so that we can have a bit longer text to read

In [7]:
del all_parts

In [8]:
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

In [9]:
import gc
# Perform garbage collection
gc.collect()

3892

In [11]:
from google.colab import files
files.download('/content/combined.wav')
#Individual pieces can also be found in this folder

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
#@title Confirm that audio same length as video. If not, keep the smallest one and cut the other (or cut them both to 20secs). This is needed for wav2lip to work
#ChatGPT Prompt: Create python code that compares an audio.wav with a video.mp4 files and if the duration of one is bigger than the other,
# it cuts the largest one to be the same duration as the smallest. If any of them is bigger than 20secs then raise an error

#Needed to avoid errors with encoding
import locale
locale.getpreferredencoding = lambda: "UTF-8"

from moviepy.editor import VideoFileClip, AudioFileClip
import os

def compare_audio_video_duration(audio_file, video_file):
    audio = AudioFileClip(audio_file)
    video = VideoFileClip(video_file)

    audio_duration = audio.duration
    video_duration = video.duration

    # Either the video or audio should be <20 secs. If any of these is larger than that, it will be cut to the duration of the other or to 20secs.
    # Might work for up to 30secs, but not guaranteed. If only video is provided, it will keep only the first 20 secs of it.

    # if audio_duration > 20 and video_duration > 20:
    #     video = video.subclip(0, 20)
    #     video.write_videofile('input_video.mp4')
    #     audio = audio.subclip(0, 20)
    #     audio.write_audiofile('input_audio.wav')

    if audio_duration != video_duration:
        min_duration = min(audio_duration, video_duration)
        if min_duration == audio_duration:
            video = video.subclip(0, min_duration)
            video.write_videofile('input_video.mp4')
            os.rename(audio_file,'input_audio.wav')
        else:
            audio = audio.subclip(0, min_duration)
            audio.write_audiofile('input_audio.wav')
            os.rename(video_file,'input_video.mp4')

    audio.close()
    video.close()

# Example usage
compare_audio_video_duration("/content/combined.wav", "/content/gdrive/MyDrive/deepfake/video_full.mp4") #input_voice.mp3

MoviePy - Writing audio in input_audio.wav


MoviePy - Done.




---



In [23]:
cd /content

/content


In [24]:
#@title <h1>Install Wav2Lip</h1>
#@markdown * Install dependencies
#@markdown * Download models
# !rm -rf /content/sample_data
# !mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
# !wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
# !wget 'https://iiitaphyd-my.sharepoint.com/:u:/g/personal/radrabha_m_research_iiit_ac_in/Eb3LEzbfuKlJiR600lQWRxgBIY27JZg80f7V9jtMfbNDaQ?e=TBFBVW' -O /content/Wav2Lip/checkpoints/wav2lip.pth
!pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
from IPython.display import clear_output
clear_output()
print("\nDone")


Done




---



In [32]:
!wget 'https://huggingface.co/Nekochu/Wav2Lip/resolve/main/wav2lip_gan.pth' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'

--2025-04-24 14:33:09--  https://huggingface.co/Nekochu/Wav2Lip/resolve/main/wav2lip_gan.pth
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.118, 18.164.174.23, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/77/6e/776eb19ab2111846cf60d972f9cc20dcc659aa064fa57f090d3a5f3e5f60b199/ca9ab7b7b812c0e80a6e70a5977c545a1e8a365a6c49d5e533023c034d7ac3d8?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27wav2lip_gan.pth%3B+filename%3D%22wav2lip_gan.pth%22%3B&Expires=1745508789&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NTUwODc4OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy83Ny82ZS83NzZlYjE5YWIyMTExODQ2Y2Y2MGQ5NzJmOWNjMjBkY2M2NTlhYTA2NGZhNTdmMDkwZDNhNWYzZTVmNjBiMTk5L2NhOWFiN2I3YjgxMmMwZTgwYTZlNzBhNTk3N2M1NDVhMWU4YTM2NWE2YzQ5ZDVlNTMzMDIzYzAzNGQ3YWMzZDg%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3Np

- Below implementation needs both audio and video to be of same length. Only specific extensions work (mp4 and wav)
- Target face in the input_video.mp4, must be "detectable" in ALL videoframes (So no black or blurry frames etc)
- wav2lip does not like very long and high res clips (1080p/30seconds recommended)
- 'Wav2Lip' model gives highly accurate lip-sync compared to 'Wav2Lip + GAN' but with inferior visual quality compared to the latter

In [33]:
!wget 'https://huggingface.co/Nekochu/Wav2Lip/resolve/main/wav2lip.pth' -O /content/Wav2Lip/checkpoints/wav2lip.pth


--2025-04-24 14:34:12--  https://huggingface.co/Nekochu/Wav2Lip/resolve/main/wav2lip.pth
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.55, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/77/6e/776eb19ab2111846cf60d972f9cc20dcc659aa064fa57f090d3a5f3e5f60b199/b78b681b68ad9fe6c6fb1debc6ff43ad05834a8af8a62ffc4167b7b34ef63c37?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27wav2lip.pth%3B+filename%3D%22wav2lip.pth%22%3B&Expires=1745508852&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NTUwODg1Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy83Ny82ZS83NzZlYjE5YWIyMTExODQ2Y2Y2MGQ5NzJmOWNjMjBkY2M2NTlhYTA2NGZhNTdmMDkwZDNhNWYzZTVmNjBiMTk5L2I3OGI2ODFiNjhhZDlmZTZjNmZiMWRlYmM2ZmY0M2FkMDU4MzRhOGFmOGE2MmZmYzQxNjdiN2IzNGVmNjNjMzc%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1d

Below is needed to fix an error in loading - Not added in the beginning due to conflict in dependencies

In [26]:
!pip install librosa==0.9.1

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/TorToiSe-2.3.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [27]:
# Clear all user-defined variables except built-in ones
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

# Perform garbage collection to free up memory
import gc
gc.collect()

34279

In [28]:
!pip install -U numpy==2.0.0 pandas scikit-learn

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/TorToiSe-2.3.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [34]:
#@title Create Wav2Lip video (using wav2lip_gan.pth) GAN
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '/content/gdrive/MyDrive/deepfake/input_video.mp4' --audio '/content/gdrive/MyDrive/deepfake/input_audio.wav' --resize_factor 2

#Use --resize_factor 2 otherwise OOM error. Use resize_factor to reduce the video resolution, as there is a chance you might get better results for lower resolution videos.
# This might be related with the model which might have been trained on low resolution faces.

Using cuda for inference.
Reading video frames...
Number of frames available for inference: 60
/content/Wav2Lip/audio.py:100: FutureWarning: Pass sr=16000, n_fft=800 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
(80, 161)
Length of mel chunks: 56
  0% 0/1 [00:00<?, ?it/s]
  0% 0/4 [00:00<?, ?it/s]
 25% 1/4 [01:53<05:39, 113.32s/it]
 50% 2/4 [01:55<01:35, 47.88s/it] 
 75% 3/4 [01:57<00:26, 26.93s/it]
100% 4/4 [02:09<00:00, 32.41s/it]
Load checkpoint from: checkpoints/wav2lip_gan.pth
Model loaded
100% 1/1 [02:16<00:00, 136.38s/it]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --en

In [35]:
#@title Play result video -  50% scaling
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()

data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [36]:
#@title Download Result.mp4 to your computer
from google.colab import files
files.download('/content/Wav2Lip/results/result_voice.mp4') #Only after the last cell is executed this will start
files.download('/content/gdrive/MyDrive/deepfake/input_audio.wav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# #@title Delete old uploaded samples & result files, so you can start over again.
# # %rm /content/sample_data/*
# %rm /content/Wav2Lip/results/*
# from IPython.display import clear_output
# clear_output()
# print("\nDone! now press X")

# **Variations to try**


In [ ]:
# #@title Create Wav2Lip video using wav2lip.pth
# !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav"  --resize_factor 2

In [ ]:
#@title Use more padding to include the chin region (you can manually edit pads dimensions viewing and changing the code)
# !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav" --pads 0 20 0 0 --resize_factor 2

In [ ]:
# #@title Play result video -  50% scaling
# from IPython.display import HTML
# from base64 import b64encode
# mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()
# data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
# HTML(f"""
# <video width="50%" height="50%" controls>
#       <source src="{data_url}" type="video/mp4">
# </video>""")

In [ ]:
#@title Download Result.mp4 to your computer
# from google.colab import files
# files.download('/content/Wav2Lip/results/result_voice.mp4')